# AMI

In [ ]:
from dotenv import load_dotenv
load_dotenv()

True

In [22]:
episode_number = 1
# episode_number = int(episode_number)
episode = 'episode' + '{:03d}'.format(episode_number)
# Set the directory for the episode
directory = 'podcast/' + episode 

In [23]:
import os
articles = os.listdir(f'podcast/{episode}/text')
articles

['Disney paiera au moins 8,6 milliards de dollars pour détenir Hulu en entier.txt',
 "Francis Mer, l'homme fort de la sidérurgie française.txt",
 "HSBC, première grande banque à mettre des lingots d'or dans la blockchain.txt",
 'Missiles _ MBDA veut créer un « hub » en Bavière.txt',
 "Sommet mondial de l'IA _ de l'unité et des dissensions.txt"]

In [14]:
article = articles[3]
article

'Missiles _ MBDA veut créer un « hub » en Bavière.txt'

In [15]:
import re

# article = "Les Echos - Missiles : MBDA veut créer un « hub » en Bavière"
article = re.sub(r'[<>:"/\\|?*]', '', article)
print(article)

Missiles _ MBDA veut créer un « hub » en Bavière.txt


In [ ]:
import re

def sanitize_filename(filename):
    # Characters to be replaced with an underscore
    invalid_characters = r'[\\/*?"<>|:]'
    # Replace invalid characters with an underscore
    sanitized = re.sub(invalid_characters, '_', filename)
    return sanitized

# Original string
original_string = "Missiles : MBDA veut créer un « hub » en Bavière"

# Sanitized string for file name
sanitized_string = sanitize_filename(original_string)

print("Original String:", original_string)
print("Sanitized Filename:", sanitized_string)

Original String: Missiles : MBDA veut créer un « hub » en Bavière
Sanitized Filename: Missiles _ MBDA veut créer un « hub » en Bavière


## Youtube

In [ ]:
%pip install pytube

  Using cached pytube-15.0.0-py3-none-any.whl (57 kB)
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
import os
os.chdir('C:\\Users\\ydebray\\Downloads\\ami')
os.getcwd()

'C:\\Users\\ydebray\\Downloads\\ami'

In [59]:
url = 'https://www.youtube.com/watch?v=ieQWjSXdv-Y'
title = 'generic'

In [1]:
url = 'https://www.youtube.com/watch?v=f4aTOBPy73w'
title = "Webinar - Développement d'application Scilab (Francais)"

In [61]:
from pytube import YouTube

def download_audio_from_youtube(url, output_filename='output'):
    yt = YouTube(url)
    
    # Grab the highest quality audio stream
    audio_stream = yt.streams.filter(only_audio=True, file_extension="mp4").first()
    
    # Download the audio stream
    audio_stream.download(filename=output_filename)

download_audio_from_youtube(url, title+'.mp3')

## OpenAI summary

In [18]:
import openai, os
openai.api_key = os.getenv("OPENAI_API_KEY")
def summarize(text):
    inst = '''Resume l'article suivant en moins de 1000 caracteres:''' 
    completion = openai.ChatCompletion.create(
        model='gpt-3.5-turbo-16k',
        messages= [
            {'role': 'system', 'content': inst },
            {'role': 'user', 'content': text }]
    )
    return completion.choices[0].message.content

In [16]:
# Read the text from the file
with open(f'podcast/{episode}/text/' + article, 'r') as f:
    text = f.read()

In [19]:
summary = summarize(text)

In [20]:
len(summary)

899

In [21]:
# Write the summary string to the file
with open(f'podcast/{episode}/summary/' + article, 'w', encoding='utf-8') as f:
    f.write(summary)

## Elevenlabs

In [34]:
from elevenlabs import generate, play, save, voices, set_api_key
set_api_key(os.getenv('ELEVENLABS_KEY'))

In [35]:
voices_list = list(voices())
voices_list

[Voice(voice_id='21m00Tcm4TlvDq8ikWAM', name='Rachel', category='premade', description=None, labels={'accent': 'american', 'description': 'calm', 'age': 'young', 'gender': 'female', 'use case': 'narration'}, samples=None, settings=VoiceSettings(stability=0.75, similarity_boost=0.75), design=None),
 Voice(voice_id='2EiwWnXFnvU5JabPnv8n', name='Clyde', category='premade', description=None, labels={'accent': 'american', 'description': 'war veteran', 'age': 'middle aged', 'gender': 'male', 'use case': 'video games'}, samples=None, settings=VoiceSettings(stability=0.5, similarity_boost=0.75), design=None),
 Voice(voice_id='AZnzlk1XvdvUeBnXmlld', name='Domi', category='premade', description=None, labels={'accent': 'american', 'description': 'strong', 'age': 'young', 'gender': 'female', 'use case': 'narration'}, samples=None, settings=VoiceSettings(stability=0.5, similarity_boost=0.75), design=None),
 Voice(voice_id='CYw3kZ02Hs0563khs1Fj', name='Dave', category='premade', description=None, la

In [69]:
voice = 'Yann'

# Generate audio for the content
audio = generate(
    text=summary,
    voice=voice,
    model="eleven_multilingual_v2"
)

save(audio,f'podcast/{episode}/audio/' + article.replace('.txt', '.mp3'))

## Polly

In [30]:
import boto3
import os
# Import the necessary libraries

# Set up the AWS credentials
aws_access_key_id = os.environ.get('AWS_ACCESS_KEY_ID')
aws_secret_access_key = os.environ.get('AWS_SECRET_ACCESS_KEY')
aws_region_name = os.environ.get('AWS_REGION_NAME')

In [32]:
# # Read the text from the file
# with open(f'podcast/{episode}/summary/' + article, 'r') as f:
#     text = f.read()

text = summary

# Initialize the Polly client
polly_client = boto3.Session(
    aws_access_key_id=aws_access_key_id,
    aws_secret_access_key=aws_secret_access_key,
    region_name=aws_region_name).client('polly')

# Split the text into chunks of less than 3000 characters
chunks = [text[i:i+3000] for i in range(0, len(text), 3000)]

# Initialize an empty byte array to hold the audio stream
audio_stream = bytearray()

for chunk in chunks:
    # Synthesize the speech using the chunk
    response = polly_client.synthesize_speech(
        Text=chunk,
        OutputFormat='mp3',
        VoiceId='Lea') # French voices, Lea, Celine, Mathieu, Chantal

    # Append the audio stream from this chunk to the overall audio stream
    audio_stream.extend(response['AudioStream'].read())

# Save the synthesized speech to an MP3 file
with open(f'podcast/{episode}/audio/' + article.replace('.txt', '.mp3'), 'wb') as f:
    f.write(audio_stream)

## Intro

In [76]:
article = 'ami_{:03d}.txt'.format(episode_number)
# Read the text from the file
with open(f'podcast/{episode}/' + article, 'r',encoding='utf-8') as f:
    text = f.read()

voice = 'Yann'

# Generate audio for the content
audio = generate(
    text=text,
    voice=voice,
    model="eleven_multilingual_v2"
)

save(audio,f'podcast/{episode}/intro.mp3')

## Edit

In [9]:
audio_files = [f for f in os.listdir(directory) if f.endswith('.mp3')]
audio_files

['Disney paiera au moins 8,6 milliards de dollars pour détenir Hulu en entier.mp3',
 "Francis Mer, l'homme fort de la sidérurgie française.mp3",
 "HSBC, première grande banque à mettre des lingots d'or dans la blockchain.mp3",
 'Missiles _ MBDA veut créer un « hub » en Bavière.mp3',
 "Sommet mondial de l'IA _ de l'unité et des dissensions.mp3"]

In [ ]:
from pydub import AudioSegment
import os

# AudioSegment.converter = 'C:\\ffmpeg\\bin'

# Set the directory containing the audio files
directory = 'podcast/episode001/audio/'

# Get a list of all the audio files in the directory
audio_files = [f for f in os.listdir(directory) if f.endswith('.mp3')]

# Initialize an empty AudioSegment object to hold the concatenated audio
concatenated_audio = AudioSegment.empty()

# Add the intro audio to the concatenated audio
intro_audio = AudioSegment.from_mp3('podcast/episode001/intro.mp3')
concatenated_audio += intro_audio

# Loop through the audio files and append them to the concatenated audio
for file in audio_files:
    audio = AudioSegment.from_mp3(directory + file)
    concatenated_audio += audio

# Export the concatenated audio to a file
concatenated_audio.export('podcast/episode001/episode001.mp3', format='mp3')
